In [1]:
from urllib.request import urlopen
import urllib
import urllib.request
import glob
import bs4
from bs4 import BeautifulSoup
import re
import bisect

In [2]:
def get_soup_object(url):
    """ given url, apply suitable user agent, get html, and then return beautifulsoup of that html"""
    
    url=url
    user_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'
    headers={'User-Agent':user_agent} 
    
    request=urllib.request.Request(url=url, headers=headers)
    html=urlopen(request)
    bs=BeautifulSoup(html)
    
    return bs

In [3]:
def get_individual_artpiece_links(url):
    """ go through the list of artworks, and then return the links of each art piece. """
    a_tags=get_soup_object(url)
    
    a_tags=a_tags.find_all("a")
    links=[]
    
    for a in a_tags:
        href=a.get('href')
        if type(href)==type("string"):
            if "http" in href:
                if "item" in href:
                    links.append(href)
    return list(set(links))

In [13]:
def get_picture_link(url):
    """access the download option, then return a link of jpeg image that has the largest size."""
    download_soup=get_soup_object(url)
    
    #download_options=download_soup.find_all("select", {"class":"select-default", "id":"select-resource1"})
    #if len(download_options)==0:
    download_options=download_soup.find_all("select", {"class":"select-default", "id":"select-resource0"})
        
    if len(download_options)==0:
        print("download option not found")
    else:
        option=download_options[0].find_all("option")

        if len(option)==0:
            print(" option not found")
        else:
            store={}
            store["value"]=[]
            store["numb"]=[]
            store["byte_type"]=[]
            store["raw_byte"]=[]

            for o in option:
                if o.get("data-file-download")=="JPEG":

                    pic=o.get("value")
                    numb=str(o).split("\xa0")[-2][1:]
                    byte_type=str(o).split("\xa0")[-1][:2]

                    store["value"].append(pic)
                    store["numb"].append(numb)
                    store["byte_type"].append(byte_type)

                    if byte_type=="KB":
                        store["raw_byte"].append(float(numb)*1024)
                    elif byte_type=="MB":
                        store["raw_byte"].append(float(numb)*1024**2)
                    else:
                        print("type not recognized: ",byte_type)

            ind=bisect.bisect_left(store["raw_byte"],max(store['raw_byte']))
            link=store["value"][ind]
            return link


In [14]:
def download_picture(url, directory, name):
    """download an image from a link"""
    
    user_agent = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'
    headers={'User-Agent':user_agent} 

    opener = urllib.request.build_opener()
    opener.addheaders=[('User-agent',user_agent)]
    urllib.request.install_opener(opener)
    urllib.request.urlretrieve(url, ".\\data\\Japanese\\%s\\%s.jpg" % (directory,name))

## Download Japanese painting

In [15]:
#assert False  # prevent accidental run

number_of_images=2662
images_per_page=100
number_of_pages=number_of_images//images_per_page

for i in range(17,number_of_pages+1): # page 15 to past second page
    url1="https://www.loc.gov/collections/japanese-fine-prints-pre-1915/"
    url2="/?c=%s&sp=%s&st=list" % (images_per_page, i)
    url=url1+url2
    artpiece_links=get_individual_artpiece_links(url)
    
    try:
        assert len(artpiece_links)==images_per_page
    except:
        print("ERROR", len(artpiece_links))
    
    for e,j in enumerate(artpiece_links):
        picture_link=get_picture_link(j)
        name=i*images_per_page+e
        download_picture(picture_link, "trainB", str(name))


download option not found


TypeError: expected string or bytes-like object

## Download Japanese American Internment

In [ ]:
#assert False  # prevent accidental run
number_of_images=258
images_per_page=25
number_of_pages=number_of_images//images_per_page
number_of_pages

for i in range(1,number_of_pages+1): # page 1 to past second page
    url1="https://www.loc.gov/collections/ansel-adams-manzanar/"
    url2="/?c=%s&sp=%s&st=list" % (images_per_page, i)
    url=url1+url2
    artpiece_links=get_individual_artpiece_links(url)
    
    try:
        assert len(artpiece_links)==images_per_page
    except:
        print("ERROR", len(artpiece_links))
    
    for e,j in enumerate(artpiece_links):
        picture_link=get_picture_link(j)
        name=i*images_per_page+e
        download_picture(picture_link, "trainA" ,str(name))